In [6]:
import psycopg2

# Load configuration
def load_config():
    import yaml
    with open('config.yaml', 'r') as file:
        return yaml.safe_load(file)

# Connect to the database
config = load_config()
db_config = config['DATABASE']
conn = psycopg2.connect(
    dbname=db_config['NAME'],
    user=db_config['USER'],
    password=db_config['PASSWORD'],
    host=db_config['HOST'],
    port=db_config['PORT']
)

# Create a cursor
cur = conn.cursor()

# Add the new column `dm_preference` if it doesn't exist
cur.execute("""
    DO $$
    BEGIN
        IF NOT EXISTS (
            SELECT 1
            FROM information_schema.columns
            WHERE table_name='birthdays' AND column_name='dm_preference'
        ) THEN
            ALTER TABLE birthdays ADD COLUMN dm_preference BOOLEAN DEFAULT FALSE;
        END IF;
    END $$;
""")

# Commit changes and close the connection
conn.commit()
cur.close()
conn.close()

# Database Management for Birthday Bot
This notebook helps manage the database operations for the birthday bot.

In [7]:
import psycopg2
import yaml

def load_config():
    with open('config.yaml', 'r') as file:
        return yaml.safe_load(file)

def get_db_connection():
    config = load_config()
    db_config = config['DATABASE']
    return psycopg2.connect(
        dbname=db_config['NAME'],
        user=db_config['USER'],
        password=db_config['PASSWORD'],
        host=db_config['HOST'],
        port=db_config['PORT']
    )

# Create a fresh connection
conn = get_db_connection()

## View current users and their DM preferences

In [10]:
try:
    with conn:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT user_id, username, birthday, dm_preference
                FROM birthdays
                ORDER BY username
            """)
            results = cur.fetchall()
            
            print("Current users and their DM preferences:")
            for row in results:
                print(f"User: {row[1]}, DM Preference: {row[3]}")
except Exception as e:
    print(f"Error: {e}")
    conn.rollback()  # Explicitly rollback on error
    # Get a fresh connection if needed
    conn = get_db_connection()

Current users and their DM preferences:
User: Abends, DM Preference: False
User: Albarn, DM Preference: False
User: AlexDieNase, DM Preference: False
User: Altdolphin, DM Preference: False
User: Bananu, DM Preference: False
User: Bubbels, DM Preference: False
User: Busten, DM Preference: False
User: D.Lightman, DM Preference: False
User: Dana, DM Preference: False
User: Der Fiebich, DM Preference: False
User: Error404, DM Preference: False
User: Error404-StreamingAudio, DM Preference: False
User: Falconslab, DM Preference: False
User: Flo/ Kneder, DM Preference: False
User: Frau Dr Bob, DM Preference: False
User: Grufnax, DM Preference: False
User: Hec, DM Preference: False
User: Holländer, DM Preference: False
User: Husky, DM Preference: False
User: Jay, DM Preference: False
User: Kasane, DM Preference: False
User: Keeper34, DM Preference: False
User: KnownAsMage, DM Preference: False
User: Lasir, DM Preference: False
User: Leviathuna, DM Preference: False
User: Markus, DM Preference:

## Initialize all DM preferences to False

In [11]:
try:
    with conn:
        with conn.cursor() as cur:
            cur.execute("""
                UPDATE birthdays
                SET dm_preference = FALSE
                WHERE dm_preference IS NULL
            """)
            rows_updated = cur.rowcount
            print(f"{rows_updated} users had their DM preferences initialized to False")
except Exception as e:
    print(f"Error: {e}")
    conn.rollback()  # Explicitly rollback on error
    conn = get_db_connection()

0 users had their DM preferences initialized to False


## Verify the changes

In [ ]:
try:
    with conn:
,
                SELECT COUNT(*) as total,
                       SUM(CASE WHEN dm_preference = TRUE THEN 1 ELSE 0 END) as dm_enabled,
                       SUM(CASE WHEN dm_preference = FALSE THEN 1 ELSE 0 END) as dm_disabled
                FROM birthdays
            """)
            stats = cur.fetchone()
            
            print(f"Total users: {stats[0]}")
            print(f"Users with DMs enabled: {stats[1]}")
            print(f"Users with DMs disabled: {stats[2]}")
except Exception as e:
    print(f"Error: {e}")
    conn.rollback()  # Explicitly rollback on error
    conn = get_db_connection()

In [12]:
# Close the connection when done
try:
    conn.close()
except:
    pass  # Connection might already be closed